In [1]:
import pandas as pd

In [16]:
df = pd.read_csv("wine.csv")

In [17]:
df.head()

,wine_id,name,food,grapes,rating_count,rating_average,label_count,review_count,body,alcohol,...,rose hip,brioche,dark fruit,butterscotch,creme fraiche,starfruit,wild iris,marjoram,cherry cough syrup,butter
0,1938520,1882 Cabernet Sauvignon,"['Beef', 'Lamb', 'Game (deer, venison)', 'Matu...",['Cabernet Sauvignon'],1697,4.1,14879,16,5.0,14.5,...,0.0,0.0,35.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,14604,Les Bessards Hermitage,"['Beef', 'Lamb', 'Game (deer, venison)', 'Poul...",['Shiraz/Syrah'],1078,4.3,5370,3,5.0,14.0,...,0.0,0.0,8.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,1930757,Patriarch Estate Grown,"['Beef', 'Lamb', 'Veal', 'Poultry']","['Cabernet Sauvignon', 'Merlot']",1072,4.6,6042,25,4.0,14.2,...,0.0,0.0,7.0,0.0,0.0,0.0,0.0,0.0,0.0,2.0
3,1564280,Merlot,"['Beef', 'Lamb', 'Veal']",['Merlot'],3577,4.3,18748,52,4.0,14.4,...,0.0,0.0,38.0,0.0,0.0,0.0,0.0,0.0,0.0,4.0
4,2576427,Cabernet Sauvignon F Block,"['Beef', 'Lamb', 'Game (deer, venison)', 'Matu...",['Cabernet Sauvignon'],115,4.4,806,1,5.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [9]:
df.describe()

,wine_id,rating_count,rating_average,label_count,review_count,body,alcohol,winery_ratings_count,winery_ratings_average,winery_labels_count,...,rose hip,brioche,dark fruit,butterscotch,creme fraiche,starfruit,wild iris,marjoram,cherry cough syrup,butter
count,5.086000e+04,50860.00000,50860.000000,5.086000e+04,50860.000000,50860.000000,50860.000000,50860.000000,50860.000000,5.086000e+04,...,50772.000000,50772.000000,50772.000000,50772.000000,50772.000000,50772.000000,50772.000000,50772.000000,50772.000000,50772.000000
mean,1.823687e+06,2027.49707,3.880781,1.314874e+04,49.137692,3.774828,8.794128,27635.669898,3.844858,1.859927e+05,...,0.027830,0.864374,4.412688,0.248759,0.006441,0.038486,0.000433,0.009277,0.008154,4.105609
std,1.532959e+06,4164.64676,0.304340,3.052208e+04,262.813187,0.907073,6.633210,51466.483632,0.302593,3.867767e+05,...,0.256013,10.791668,13.391204,1.633878,0.084311,0.257437,0.021738,0.103191,0.093159,28.340527
min,1.000000e+01,1.00000,0.000000,8.000000e+00,0.000000,0.797518,0.000000,0.000000,0.000000,0.000000e+00,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,1.149261e+06,479.00000,3.700000,2.283000e+03,5.000000,3.000000,0.000000,3410.000000,3.700000,1.805400e+04,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
50%,1.380368e+06,950.00000,3.900000,5.180000e+03,11.000000,4.000000,13.000000,8852.500000,3.800000,4.801000e+04,...,0.000000,0.000000,1.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
75%,2.125150e+06,2004.00000,4.100000,1.232800e+04,29.000000,4.000000,14.000000,25568.000000,4.000000,1.505260e+05,...,0.000000,0.000000,4.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,2.000000
max,8.564957e+06,148911.00000,4.800000,1.137754e+06,22865.000000,7.349663,135.000000,540005.000000,5.000000,3.975826e+06,...,13.000000,1125.000000,516.000000,97.000000,2.000000,12.000000,2.000000,4.000000,3.000000,2715.000000


In [18]:
df.shape

(50860, 562)

In [19]:
df.columns

Index(['wine_id', 'name', 'food', 'grapes', 'rating_count', 'rating_average',
       'label_count', 'review_count', 'body', 'alcohol',
       ...
       'rose hip', 'brioche', 'dark fruit', 'butterscotch', 'creme fraiche',
       'starfruit', 'wild iris', 'marjoram', 'cherry cough syrup', 'butter'],
      dtype='object', length=562)

In [26]:
pd.set_option('display.max_columns',None)

In [30]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10 entries, 0 to 9
Columns: 562 entries, wine_id to butter
dtypes: float64(514), int64(43), object(5)
memory usage: 44.0+ KB


In [36]:
df.iloc[:,:50].isnull().sum()

wine_id                        0
name                           0
food                        1484
grapes                       500
rating_count                   0
rating_average                 0
label_count                    0
review_count                   0
body                           0
alcohol                        0
winery_ratings_count           0
winery_ratings_average         0
winery_labels_count            0
winery_wines_count             0
Aperitif                       0
Appetizers and snacks          0
Blue cheese                    0
Cured Meat                     0
Fruity desserts                0
Game                           0
Goat cheese                    0
Lamb                           0
Lean fish                      0
Mature and hard cheese         0
Mild and soft cheese           0
Mushrooms                      0
Pasta                          0
Pork                           0
Poultry                        0
Rich fish                      0
Shellfish 